# Explore here

In [ ]:
#Paso 1: Definición del problema
#🎯 Problema de negocio
#La depresión es uno de los trastornos mentales más prevalentes a nivel mundial. Se estima que millones de personas la padecen, y muchas de ellas no reciben el tratamiento adecuado debido a estigmas sociales, falta de diagnóstico o acceso limitado a servicios de salud mental. La detección temprana es esencial para intervenir oportunamente y prevenir consecuencias graves como el deterioro en la calidad de vida o incluso el suicidio.

#El objetivo de este proyecto es desarrollar un sistema de predicción que, basándose en información sociodemográfica, historial médico y hábitos de vida de los individuos, sea capaz de detectar indicios de depresión. Esto permitiría generar herramientas de apoyo para psicólogos, profesionales de salud, instituciones educativas o incluso en ambientes laborales, facilitando intervenciones tempranas y preventivas.

#🤖 Problema de Machine Learning
#Se plantea un problema de clasificación binaria, donde a partir de variables como la edad, estado civil, nivel educativo, hábitos alimenticios, patrones de sueño, historial médico, entre otras, se busca predecir si una persona se encuentra en un estado depresivo (1) o no (0).



In [ ]:
#Paso 2: Obtencion y carga del conjunto de datos

In [13]:
import pandas as pd

df = pd.read_csv('../data/raw/depression_data.csv')
print(df.head())


               Name  Age Marital Status    Education Level  \
0  Christine Barker   31        Married  Bachelor's Degree   
1  Jacqueline Lewis   55        Married        High School   
2    Shannon Church   78        Widowed    Master's Degree   
3    Charles Jordan   58       Divorced    Master's Degree   
4      Michael Rich   18         Single        High School   

   Number of Children Smoking Status Physical Activity Level  \
0                   2     Non-smoker                  Active   
1                   1     Non-smoker               Sedentary   
2                   1     Non-smoker               Sedentary   
3                   3     Non-smoker                Moderate   
4                   0     Non-smoker               Sedentary   

  Employment Status     Income Alcohol Consumption Dietary Habits  \
0        Unemployed   26265.67            Moderate       Moderate   
1          Employed   42710.36                High      Unhealthy   
2          Employed  125332.79     

In [ ]:
#Nota sobre la variable objetivo:
#Dado que el dataset original no incluía una columna explícita que indicara si una persona se encuentra en estado depresivo,
# se generó una variable sintética Depressed basada en criterios clínicamente reconocidos como factores de riesgo de depresión,
# tales como: historial de enfermedad mental, trastornos del sueño, abuso de sustancias, antecedentes familiares, entre otros.
# Esta aproximación se utilizó exclusivamente para fines académicos, con el objetivo de demostrar el proceso completo de Machine
# Learning aplicado a datos de salud mental.

#Depressed = 1 → indica un caso considerado con alto riesgo de depresión.
#Depressed = 0 → indica un caso considerado sin riesgo significativo de depresión.

In [ ]:
def calcular_depresion(row):
    factores = 0
    if row['History of Mental Illness'] == 'Yes':
        factores += 1
    if row['History of Substance Abuse'] == 'Yes':
        factores += 1
    if row['Family History of Depression'] == 'Yes':
        factores += 1
    if row['Sleep Patterns'] == 'Poor':
        factores += 1
    if row['Chronic Medical Conditions'] == 'Yes':
        factores += 1
    if row['Physical Activity Level'] == 'Sedentary':
        factores += 1
    if row['Alcohol Consumption'] == 'High':
        factores += 1
    return 1 if factores >= 2 else 0

df['Depressed'] = df.apply(calcular_depresion, axis=1)


In [ ]:
#Paso 3: Almacenar la información



In [23]:
import sqlite3
import pandas as pd

#Conectar df a SQL
conn = sqlite3.connect("depression_data.db")

# Almacenar los datos en la base de datos SQLite en una tabla llamada 'health_data'
df.to_sql('health_data', conn, if_exists='replace', index=False)
print("Datos cargados correctamente en la base de datos 'mental_health.db'")


Datos cargados correctamente en la base de datos 'mental_health.db'


In [24]:
# 1: Ver las primeras 5 filas
query1 = "SELECT * FROM health_data LIMIT 5"
print("▶ Consulta 1: Primeras 5 filas")
print(pd.read_sql(query1, conn), "\n")



▶ Consulta 1: Primeras 5 filas
               Name  Age Marital Status    Education Level  \
0  Christine Barker   31        Married  Bachelor's Degree   
1  Jacqueline Lewis   55        Married        High School   
2    Shannon Church   78        Widowed    Master's Degree   
3    Charles Jordan   58       Divorced    Master's Degree   
4      Michael Rich   18         Single        High School   

   Number of Children Smoking Status Physical Activity Level  \
0                   2     Non-smoker                  Active   
1                   1     Non-smoker               Sedentary   
2                   1     Non-smoker               Sedentary   
3                   3     Non-smoker                Moderate   
4                   0     Non-smoker               Sedentary   

  Employment Status     Income Alcohol Consumption Dietary Habits  \
0        Unemployed   26265.67            Moderate       Moderate   
1          Employed   42710.36                High      Unhealthy   
2   

In [ ]:
# 2: Contar personas deprimidas (1) vs no deprimidas (0)
query2 = "SELECT Depressed, COUNT(*) as Total FROM health_data GROUP BY Depressed"
print("▶ Consulta 2: Conteo deprimidos vs no")
print(pd.read_sql(query2, conn), "\n")



▶ Consulta 2: Conteo deprimidos vs no
   Depressed   Total
0          0  128135
1          1  285633 



In [26]:
# Consulta 3: Ver distribución por nivel de actividad física
query3 = "SELECT [Physical Activity Level], COUNT(*) as Total FROM health_data GROUP BY [Physical Activity Level]"
print("▶ Consulta 3: Conteo por nivel de actividad física")
print(pd.read_sql(query3, conn), "\n")

▶ Consulta 3: Conteo por nivel de actividad física
  Physical Activity Level   Total
0                  Active   78905
1                Moderate  158013
2               Sedentary  176850 



In [27]:
query4 = "SELECT [Sleep Patterns], COUNT(*) as Total FROM health_data GROUP BY [Sleep Patterns]"
print("▶ Consulta 4: Conteo por patrón de sueño")
print(pd.read_sql(query4, conn), "\n")

▶ Consulta 4: Conteo por patrón de sueño
  Sleep Patterns   Total
0           Fair  196789
1           Good   87397
2           Poor  129582 



In [28]:
# Consulta 5: Porcentaje promedio de depresión por consumo de alcohol
query5 = """
    SELECT [Alcohol Consumption], ROUND(AVG(Depressed), 2) as Porcentaje_Deprimidos
    FROM health_data
    GROUP BY [Alcohol Consumption]
"""
print("▶ Consulta 5: Porcentaje de deprimidos por consumo de alcohol")
print(pd.read_sql(query5, conn), "\n")

# Cerrar conexión
conn.close()

▶ Consulta 5: Porcentaje de deprimidos por consumo de alcohol
  Alcohol Consumption  Porcentaje_Deprimidos
0                High                   0.94
1                 Low                   0.61
2            Moderate                   0.61 

